In [1]:
import dask.dataframe as dd
import boto3
import os 
s3 = boto3.resource('s3', aws_access_key_id = os.environ['ACCESS'], aws_secret_access_key=os.environ['SECRET'])
my_bucket = s3.Bucket('sar-store')


paths = ["s3://" + object.bucket_name + '/' + object.key for object in my_bucket.objects.filter(Prefix='plates/')]
paths[1:]

['s3://sar-store/plates/xaa.csv',
 's3://sar-store/plates/xab1.csv',
 's3://sar-store/plates/xac2.csv',
 's3://sar-store/plates/xad.csv',
 's3://sar-store/plates/xae.csv',
 's3://sar-store/plates/xaf.csv',
 's3://sar-store/plates/xag.csv',
 's3://sar-store/plates/xah.csv',
 's3://sar-store/plates/xai.csv',
 's3://sar-store/plates/xaj.csv',
 's3://sar-store/plates/xak.csv',
 's3://sar-store/plates/xal.csv',
 's3://sar-store/plates/xam.csv',
 's3://sar-store/plates/xan.csv',
 's3://sar-store/plates/xao.csv',
 's3://sar-store/plates/xap.csv',
 's3://sar-store/plates/xaq.csv',
 's3://sar-store/plates/xar.csv',
 's3://sar-store/plates/xas.csv',
 's3://sar-store/plates/xat.csv',
 's3://sar-store/plates/xau.csv',
 's3://sar-store/plates/xav.csv',
 's3://sar-store/plates/xaw.csv',
 's3://sar-store/plates/xax.csv',
 's3://sar-store/plates/xay.csv',
 's3://sar-store/plates/xaz.csv',
 's3://sar-store/plates/xba.csv',
 's3://sar-store/plates/xbb.csv',
 's3://sar-store/plates/xbc.csv',
 's3://sar-s

In [2]:
df = dd.read_csv(paths[1:], storage_options={'anon': True})

In [3]:
df.drop_duplicates(inplace=True)

,Plate,State,License Type,Summons Number,Issue Date,Violation Time,Violation,Judgment Entry Date,Fine Amount,Penalty Amount,Interest Amount,Reduction Amount,Payment Amount,Amount Due,Precinct,County,Issuing Agency,Violation Status,Summons Image
npartitions=1,,,,,,,,,,,,,,,,,,,
,object,object,object,int64,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
len(df)

/opt/conda/lib/python3.6/site-packages/dask/local.py:290: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  result = _execute_task(task, data)


28672056

In [ ]:
types = df.groupby(['License Type'])['Summons Number'].count().compute()

In [ ]:
types.sort_values(ascending=False).head(10)

In [ ]:
vanities = df[df['License Type'] == 'SRF'].groupby('Plate')['Summons Number'].count().compute()

In [ ]:
summons_per_plate = df.groupby(['Plate', 'License Type'])['Summons Number'].count().compute()

In [6]:
PAS = df.loc[(df['License Type'].isin(['PAS', 'SRF'])) & (df.State == 'NY')]

In [7]:
PAS['type_1'] = PAS.Plate.str.match(pat = '^\w{3}\B\d{4}$') # ABC-1234
PAS['type_2'] = PAS.Plate.str.match(pat = '^d{6}$') # 123456


PAS['type_3'] = PAS.Plate.str.match(pat = '^\w{1}\B\d{3}\w{2}$')# A12 3BC
PAS['type_4'] = PAS.Plate.str.match(pat = '^\w{1}\B\d{1}\w{1}\d{3}$') # A1B 234
PAS['type_5'] = PAS.Plate.str.match(pat = '^\w{2}\B\d{3}\w{1}$')# AB1 23C
PAS['type_6'] =  PAS.Plate.str.match(pat = '^\d{4}\B\w{3}$') # 1234-ABC
PAS['type_7'] =  PAS.Plate.str.match(pat = '^\w{3}\B\d{3}$') # ABC 123
PAS['type_8'] =  PAS.Plate.str.match(pat = '^\d{1}\w{2}\d{3}$') # 1AB 234
PAS['tlc'] = PAS.Plate.str.match(pat = '^\w{1}\d{6}\w{1}$')  #T702680C


PAS['Standard_Plate'] = PAS.type_1 | PAS.type_2 | PAS.type_3 | PAS.type_4 | PAS.type_5 | PAS.type_6 | PAS.type_7 | PAS.type_8 | PAS.tlc

In [ ]:
plate_types = PAS.groupby(['Plate','Standard_Plate'])['Summons Number'].count().compute()

In [ ]:
plate_types.reset_index().groupby('Standard_Plate').mean()

In [8]:
PAS['year'] = PAS['Issue Date'].str.slice(start=6, stop = 10)

In [ ]:
grp = PAS.groupby(['Standard_Plate', 'Plate','year'])['Summons Number'].count().compute()

/opt/conda/lib/python3.6/site-packages/dask/local.py:290: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  result = _execute_task(task, data)


In [12]:
grp.head(20)

Standard_Plate  Plate     year
False           08PEPE    2017     2
                0DRAMA    2013     4
                0EM1ISH   2016     4
                0INYC     2015     8
                0NEIGHTY  2015     3
                101UMAD   2017     1
                108RX     2015     3
                10NT1ME   2017     6
                11111TAT  2017     3
                1111F     2017    17
                117VN     2017     2
                125DCH    2016     3
                136RN     2017     5
                147HOGS   2016     8
                15ONYC    2017     3
                160CAP    2017    59
                          2018    31
                168CAP    2017     5
                168HANDS  2017     9
                170COZ    2017     1
Name: Summons Number, dtype: int64

In [17]:
%matplotlib inline
grp.reset_index().groupby(['Standard_Plate', 'year']).mean()

Summons Number
Standard_Plate year                
False          2000        1.000000
               2008        2.051020
               2009        1.991957
               2010        1.959153
               2011        1.882199
               2012        1.692503
               2013        1.835600
               2014        1.405040
               2015        1.479164
               2016        2.174809
               2017        2.300544
               2018        2.059941
               2019        1.000000
               2020        1.000000
               2021        1.000000
               2022        1.000000
               2023        1.000000
               2025        1.000000
               2027        1.000000
               2028        1.000000
               2029        1.000000
               2030        1.000000
               2031        1.000000
               2046        1.000000
True           2000        1.000000
               2001        1.250000
               2003        1.000000
               2004        1.000000
               2005        1.000000
               2006        1.050000
...                             ...
               2050        1.000000
               2052        1.000000
               2055        1.000000
               2061        1.000000
               2062        1.000000
               2063        1.000000
               2066        1.000000
               2067        1.000000
               2069        1.000000
               2070        1.000000
               2071        1.000000
               2073        1.000000
               2074        1.000000
               2075        1.000000
               2076        1.000000
               2077        1.000000
               2078        1.000000
               2080        1.000000
               2083        1.000000
               2085        1.000000
               2087        1.000000
               2089        1.000000
               2090        1.000000
               2091        1.000000
               2093        1.000000
               2094        1.000000
               2095        1.200000
               2096        1.000000
               2097        1.000000
               2098        1.000000

[97 rows x 1 columns]